<a href="https://colab.research.google.com/github/robertkrutsch/alexandria/blob/main/cvf_build_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
'''
Go on the cvf page 'https://openaccess.thecvf.com/menu' and download all pdfs from the conferences.
The idea here is that all this pdfs in a workshop are similar, they are on the same umbrela and while they might have different directions , methods and datasets they are relevant to each other.
The recomandation system should be able to detect this relevance and recomand such papers. 
The script here starts to build a dataset for alexandria, the world's library!
'''
import requests
from bs4 import BeautifulSoup
import os
import urllib.request
import shutil

def creat_dir(name): #create a directory
  try:
    os.mkdir(name)
  except:
    print('dir:' + name + ' exists!')
  return

def list_files(startpath): #list all contents in a directory
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

def remove(path): #delete all contents
    """ param <path> could either be relative or absolute. """
    if os.path.isfile(path) or os.path.islink(path):
        os.remove(path)  # remove the file
    elif os.path.isdir(path):
        shutil.rmtree(path)  # remove dir and all contains
    else:
        raise ValueError("file {} is not a file or dir.".format(path))

def GetConferenceURL(conference):
  tmp = conference.find_all('a')
  conf_field = tmp.pop(1)
  conf_href = conf_field.get('href')
  URL = conf_href[0:len(conf_href)-3]
  conference_name = conf_href[1:len(conf_href)-18]
  return URL,conference_name


def GetWorkshopURL(workshop):
  tmp = workshop.find_all('a')
  conf_field = tmp.pop(0)
  workshop_name = conf_field.get_text()
  conf_href = conf_field.get('href')
  URL = conf_href[0:len(conf_href)-3]
  return URL, workshop_name

def OpenURL(URL):
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  results = soup.find(id='content')
  return results

remove('/content/dataset')
creat_dir('dataset')
start_page = 'https://openaccess.thecvf.com/menu'
content = OpenURL(start_page)
#find all conferences
conf_list = content.find_all('dd')


#parse all conferences
dataset = 1
for conference in conf_list:
  URL_Conference, Conference_Name = GetConferenceURL(conference)  
  URL_Conference = start_page[0:len(start_page)-4] + URL_Conference 
  content_conference = OpenURL(URL_Conference) 
  workshop_list = content_conference.find_all('dd')
  for workshop in workshop_list:
    URL_Workshop, Name_Workshop = GetWorkshopURL(workshop)
    URL_Workshop = URL_Conference[0:len(URL_Conference)-4] + URL_Workshop
    content_workshop = OpenURL(URL_Workshop)

    papers_list = content_workshop.find_all('dd')

    # need to parse for all papers urls and then download them in some directory 
    # after I need some sort of 

    Name_Workshop = Name_Workshop.replace("/","_") #some have a slash inside and that breaks the directory structure
    workshop_dir = '/content/dataset/'+ Conference_Name + "_"+ Name_Workshop.replace(" ","")
    creat_dir(workshop_dir)
    for paper in papers_list:
      tmp = paper.find_all('a')
      for t in tmp:
        if(t.get_text()=='pdf'):
          paper_path  =   t.get('href')
          paper_path  =  start_page[0:len(start_page)-4] + paper_path[3:len(paper_path)]
          #print(paper_path,'\n')

          download_url = paper_path.replace(" ","%20")
          urllib.request.urlretrieve(download_url, workshop_dir + "/" + paper_path[paper_path.rfind("/")+1:len(paper_path)]) #copy the pdf to the directory

# Finally remove some of the non-populated things that might not conform to all the handcrafted filters up there
# The filters in the code are built by inspecting the page that might be subject to change over time, so once you see this code all things might not be working...
root = '/content/dataset'
folders = list(os.walk(root))[1:]

for folder in folders:
    # folder example: ('FOLDER/3', [], ['file'])
    if not folder[2]:
        os.rmdir(folder[0])        
list_files('/content/dataset')


dataset/
    CVPR2018_WomeninComputerVision/
        Islam_Early_Diagnosis_of_CVPR_2018_paper.pdf
        Tautkute_I_Know_How_CVPR_2018_paper.pdf
        Bazazian_Word_Spotting_in_CVPR_2018_paper.pdf
        Yamagishi_Cosmetic_Features_Extraction_CVPR_2018_paper.pdf
        Singh_Encapsulating_the_Impact_CVPR_2018_paper.pdf
        Cornia_SAM_Pushing_the_CVPR_2018_paper.pdf
        Zheng_RPIfield_A_New_CVPR_2018_paper.pdf
        Demir_A_Holistic_Framework_CVPR_2018_paper.pdf
        Malhotra_Autonomous_Detection_of_CVPR_2018_paper.pdf
        Bittner_Automatic_Large-Scale_3D_CVPR_2018_paper.pdf
        Behpour_ARC_Adversarial_Robust_CVPR_2018_paper.pdf
        Demir_WiCV_2018_The_CVPR_2018_paper.pdf
        Gajic_Cross-Domain_Fashion_Image_CVPR_2018_paper.pdf
        Iqbal_Discovering_Style_Trends_CVPR_2018_paper.pdf
        Timm_Large-Scale_Ecological_Analyses_CVPR_2018_paper.pdf
        Gurel_Towards_More_Accurate_CVPR_2018_paper.pdf
    ICCV2017_AssistiveComputerVisionandRobotics/


In [84]:
'''
How much data do we have in this dataset ?
'''
total_size = 0
start_path = '.'  # To get size of current directory
for path, dirs, files in os.walk(start_path):
    for f in files:
        fp = os.path.join(path, f)
        total_size += os.path.getsize(fp)
print("Directory size: " + str(total_size/1024/1024/1024)+"GB")

Directory size: 5.742902730591595GB


In [85]:
'''
Now let's make an archive of all this papers and save them locally for backup or local training.
'''

!zip -r /content/cvf_dataset.zip /content/dataset

from google.colab import files
files.download("/content/cvf_dataset.zip")


  adding: content/dataset/ (stored 0%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/ (stored 0%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Islam_Early_Diagnosis_of_CVPR_2018_paper.pdf (deflated 30%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Tautkute_I_Know_How_CVPR_2018_paper.pdf (deflated 31%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Bazazian_Word_Spotting_in_CVPR_2018_paper.pdf (deflated 17%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Yamagishi_Cosmetic_Features_Extraction_CVPR_2018_paper.pdf (deflated 18%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Singh_Encapsulating_the_Impact_CVPR_2018_paper.pdf (deflated 19%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Cornia_SAM_Pushing_the_CVPR_2018_paper.pdf (deflated 8%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Zheng_RPIfield_A_New_CVPR_2018_paper.pdf (deflated 31%)
  adding: content/dataset/CVPR2018_WomeninComputerVision/Demir_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>